In [2]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables show all columns
pd.set_option("max_columns", 100)

In [2]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [3]:
url = "https://steamspy.com/api.php"
parameters = {"request": "all"}

steam_spy_all = pd.DataFrame()

for page in range (0, 5):
    # request 'all' from steam spy and parse into dataframe
    parameters['page'] = page
    json_data = get_request(url, parameters=parameters)
    steam_spy_all = steam_spy_all.append(
        pd.DataFrame.from_dict(json_data, orient='index')
    )

# generate sorted app_list from steamspy data
app_list = steam_spy_all.reset_index(drop=True)
# app_list = steam_spy_all.sort_values('appid').reset_index(drop=True)

# export disabled to keep consistency across download sessions
app_list.to_csv('data/download/app_list.csv', index=False)

# instead read from stored csv
app_list = pd.read_csv('data/download/app_list.csv')

# display first few rows
app_list.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,570,Dota 2,Valve,Valve,NaN,1248293,233209,0,"100,000,000 .. 200,000,000",35068,1406,1135,532,0,0,0,587722
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,NaN,4810746,647995,0,"50,000,000 .. 100,000,000",28779,950,6616,394,0,0,0,1126980
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",NaN,918643,778244,0,"50,000,000 .. 100,000,000",25244,760,11603,279,2999,2999,0,431349
3,440,Team Fortress 2,Valve,Valve,NaN,723875,45972,0,"50,000,000 .. 100,000,000",7853,1067,366,257,0,0,0,107470
4,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,NaN,390800,37429,0,"20,000,000 .. 50,000,000",6484,3017,345,895,0,0,0,25223


In [248]:
def get_app_data(start, stop, parser, pause, app_list):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause, app_list)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

In [5]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [249]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data


# Set file parameters
download_path = 'data/download'
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

# steam_columns = [
#     'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
#     'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
#     'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
#     'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
#     'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
#     'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
#     'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
#     'background', 'content_descriptors'
# ]

steam_columns = [
    'type',
    'owners',
    'name',
    'steam_appid',
    'required_age',
    'is_free',
    'controller_support',
    'dlc',
    'developers',
    'publishers',
    'demos',
    'price_overview',
    'platforms',
    'metacritic',
    'categories',
    'genres',
    'recommendations',
    'release_date'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=newlist,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=100,
    batchsize=10
)

Starting at index 0:

Exported lines 0-9 to steam_app_data.csv. Batch 0 time: 0:00:17 (avg: 0:00:17, remaining: 0:02:35)
Exported lines 10-19 to steam_app_data.csv. Batch 1 time: 0:00:18 (avg: 0:00:17, remaining: 0:02:19)
Exported lines 20-29 to steam_app_data.csv. Batch 2 time: 0:00:17 (avg: 0:00:17, remaining: 0:02:01)
Exported lines 30-39 to steam_app_data.csv. Batch 3 time: 0:00:17 (avg: 0:00:17, remaining: 0:01:44)
Exported lines 40-49 to steam_app_data.csv. Batch 4 time: 0:00:13 (avg: 0:00:16, remaining: 0:01:22)
Exported lines 50-59 to steam_app_data.csv. Batch 5 time: 0:00:02 (avg: 0:00:14, remaining: 0:00:56)
Exported lines 60-69 to steam_app_data.csv. Batch 6 time: 0:00:02 (avg: 0:00:12, remaining: 0:00:36)
Exported lines 70-79 to steam_app_data.csv. Batch 7 time: 0:00:02 (avg: 0:00:11, remaining: 0:00:22)
Exported lines 80-89 to steam_app_data.csv. Batch 8 time: 0:00:02 (avg: 0:00:10, remaining: 0:00:10)
Exported lines 90-99 to steam_app_data.csv. Batch 9 time: 0:00:02 (avg:

In [7]:
from urllib.parse import urlparse, urldefrag, urljoin
from urllib.request import urlopen
from url_normalize import url_normalize
from bs4 import BeautifulSoup
from queue import Queue
import time

In [8]:
SEED = "https://simple.wikipedia.org/wiki/Data_science"

def download_from_the_internet(url):
    '''Скачивает сраницу с интернета

    Параметры:
        url (str) - ссылка на страницу для скачивания

    Возвращает:
        str - html-страница в виде строки, None в случае неудачи
    '''
    try:
        return urlopen(url).read().decode('utf-8')
    except KeyboardInterrupt:
        raise
    except:
        return None

    
def extract_links_from_html(url, html):
    '''Парсит ссылки на странице

    Принимает:
        url (str) - исходный урл страницы
        html (str) - содержание html-страницы

    Возвращает:
        list - список ссылок, находящихся на странице
    '''
    parser = BeautifulSoup(html)
    # Формируем ссылки на те страницы, на которые ссылается документ
    return [urljoin(url, link.get('href')) for link in parser.findAll('a')]


def extract_text_info_from_html(html):
    '''Парсит текстовую информацию на страницу

    Принимает:
         html (str) - содержание html-страницы

    Возвращает:
        dict - текстовая часть страницы по ключу text,
               название по ключу title
    '''
    soup = BeautifulSoup(html, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()
    
    # Объединяем строки текста
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    # Находим название на странице
    title = soup.find('title').string
    
    return {'text': text, 'title': title}
    

def is_wiki_article(url):
    '''Проверяет, является ли ссылка страницей на simple.wikipedia'''
    if 'simple.wikipedia' not in url:
        return False
    
    prohibited = ['Category', 'Special', 'Wikipedia',
                  'php', 'Help']
    
    # Проверяем, есть ли запрещённые строки в ссылке
    for token in prohibited:
        if token in url:
            return False
        
    return True

In [9]:
wiki_df_json = []

# Максимальное число загруженных страниц
MAX_DOWNLOADS = 10
# Создаём очередь для обхода в ширину
q = Queue()
q.put(SEED)

already_visited = set()
n_downloads = 0
time_start = time.time()

while not q.empty():
    # Нормализуем урл
    main_url = url_normalize(q.get())
    if main_url in already_visited:
        continue
    already_visited.add(main_url)
    html = download_from_the_internet(main_url)
    
    # Извлекаем ссылки из страницы
    children_links = extract_links_from_html(main_url, html)
    time.sleep(0.5)
    
    # Извлекаем текст страницы
    text_info = extract_text_info_from_html(html) 
    
    # Добавляем запись в таблицу
    text_info['url'] = main_url
    wiki_df_json.append(text_info)
    
    n_downloads += 1
    if n_downloads > MAX_DOWNLOADS:
        break
    
    # Добавляем ещё не посещённые ссылки в очередь
    for child in children_links:
        if url_normalize(child) not in already_visited and is_wiki_article(child):
            q.put(child)

In [10]:
wiki_df = pd.DataFrame(wiki_df_json)

In [11]:
wiki_df.sample(5)

,text,title,url
9,"Machine learning - Simple English Wikipedia, t...","Machine learning - Simple English Wikipedia, t...",https://simple.wikipedia.org/wiki/Machine_lear...
6,"Signal processing - Simple English Wikipedia, ...","Signal processing - Simple English Wikipedia, ...",https://simple.wikipedia.org/wiki/Signal_proce...
5,"Data - Simple English Wikipedia, the free ency...","Data - Simple English Wikipedia, the free ency...",https://simple.wikipedia.org/wiki/Data
1,"Data science - Simple English Wikipedia, the f...","Data science - Simple English Wikipedia, the f...",https://simple.wikipedia.org/wiki/Data_science...
7,"Mathematics - Simple English Wikipedia, the fr...","Mathematics - Simple English Wikipedia, the fr...",https://simple.wikipedia.org/wiki/Mathematics


In [65]:
main_url = 'data/download/English - Language Stats - SteamSpy - All the data and stats about Steam games.html'
html_file = open(main_url, 'r', encoding='utf-8')
html = html_file.read()
rows = html.split('<tr>')

owners_cnt = pd.DataFrame(columns=['name', 'owners'])

for i in range (2, 29659):
    if i % 100 == 0:
        print(i)
    soup = BeautifulSoup(rows[i], features="html.parser")
    cells = soup.find_all('td')
    new_row = pd.DataFrame(
        data=[[cells[1]['data-order'], cells[5]['data-order']]],
        columns=['name', 'owners']
    )
    owners_cnt = owners_cnt.append(new_row)

owners_cnt

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

IndexError: list index out of range

In [66]:
owners_cnt

,name,owners
0,BioShock,4174000
0,Half-Life,7956000
0,Half-Life 2,11441000
0,Grand Theft Auto V,32986000
0,Portal 2,12114000
...,...,...
0,Undead Development,12000
0,Distant Space,104000
0,Million Dungeon,12000
0,Escape the Clinic,69000


In [67]:
app_list = pd.read_csv('data/download/app_list2.csv')
app_data = pd.read_csv('data/download/steam_app_data2.csv')

app_data

,type,name,steam_appid,required_age,is_free,controller_support,dlc,developers,publishers,demos,price_overview,platforms,metacritic,categories,genres,recommendations,release_date
0,game,Counter-Strike,10,0.0,False,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 25900, 'final':...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 103953},"{'coming_soon': False, 'date': '1 Nov, 2000'}"
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 3915},"{'coming_soon': False, 'date': '1 Apr, 1999'}"
2,game,Day of Defeat,30,0.0,False,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 2859},"{'coming_soon': False, 'date': '1 May, 2003'}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 1346},"{'coming_soon': False, 'date': '1 Jun, 2001'}"
4,game,Half-Life: Opposing Force,50,0.0,False,NaN,NaN,['Gearbox Software'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",{'total': 8970},"{'coming_soon': False, 'date': '1 Nov, 1999'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,game,Omega-Altro,1550800,0.0,False,NaN,NaN,['Rakun'],['Rakun Games'],NaN,"{'currency': 'RUB', 'initial': 10200, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '1 Mar, 2021'}"
4996,game,Rifle Strike,1553420,0.0,True,NaN,NaN,"['Štěpán Sedmík', 'Lukáš Vorlíček']",['Jakub Sedmík'],NaN,NaN,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '19 Mar, 2021'}"
4997,game,OMFG: One Million Fatal Guns,1555710,0.0,True,NaN,NaN,['Gun Hit Wonders'],['DigiPen Institute of Technology'],NaN,NaN,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '11 Mar, 2021'}"
4998,game,Wonhon: Prologue,1557060,0.0,True,full,NaN,['BUSAN SANAI GAMES'],['Super.com'],NaN,NaN,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '28 Feb, 2021'}"


In [76]:
app_data = app_data.merge(
    owners_cnt,
    how='left',
    on='name'
)

In [83]:
app_data = app_data.merge(
    app_list,
    how='left',
    left_on='steam_appid',
    right_on='appid'
)

,type,name_x,steam_appid,required_age,is_free,controller_support,dlc,developers,publishers,demos,price_overview,platforms,metacritic,categories,genres,recommendations,release_date,owners_x,appid,name_y,developer,publisher,score_rank,positive,negative,userscore,owners_y,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,game,Counter-Strike,10,0.0,False,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 25900, 'final':...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 103953},"{'coming_soon': False, 'date': '1 Nov, 2000'}",18566000,10.0,Counter-Strike,Valve,Valve,NaN,174744.0,4487.0,0.0,"10,000,000 .. 20,000,000",9273.0,377.0,295.0,156.0,999.0,999.0,0.0,19625.0
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 3915},"{'coming_soon': False, 'date': '1 Apr, 1999'}",NaN,20.0,Team Fortress Classic,Valve,Valve,NaN,4828.0,821.0,0.0,"2,000,000 .. 5,000,000",49.0,0.0,20.0,0.0,499.0,499.0,0.0,89.0
2,game,Day of Defeat,30,0.0,False,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 2859},"{'coming_soon': False, 'date': '1 May, 2003'}",6159000,30.0,Day of Defeat,Valve,Valve,NaN,4647.0,515.0,0.0,"5,000,000 .. 10,000,000",954.0,0.0,32.0,0.0,499.0,499.0,0.0,141.0
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 1346},"{'coming_soon': False, 'date': '1 Jun, 2001'}",NaN,40.0,Deathmatch Classic,Valve,Valve,NaN,1712.0,372.0,0.0,"5,000,000 .. 10,000,000",649.0,13.0,8.0,13.0,499.0,499.0,0.0,6.0
4,game,Half-Life: Opposing Force,50,0.0,False,NaN,NaN,['Gearbox Software'],['Valve'],NaN,"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",{'total': 8970},"{'coming_soon': False, 'date': '1 Nov, 1999'}",8229000,50.0,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,10834.0,558.0,0.0,"5,000,000 .. 10,000,000",781.0,0.0,139.0,0.0,499.0,499.0,0.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5009,game,Omega-Altro,1550800,0.0,False,NaN,NaN,['Rakun'],['Rakun Games'],NaN,"{'currency': 'RUB', 'initial': 10200, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '1 Mar, 2021'}",NaN,1550800.0,Omega-Altro,Rakun,Rakun Games,NaN,1.0,0.0,0.0,"100,000 .. 200,000",0.0,0.0,0.0,0.0,199.0,399.0,50.0,0.0
5010,game,Rifle Strike,1553420,0.0,True,NaN,NaN,"['Štěpán Sedmík', 'Lukáš Vorlíček']",['Jakub Sedmík'],NaN,NaN,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '19 Mar, 2021'}",110000,1553420.0,Rifle Strike,"Štěpán Sedmík, Lukáš Vorlíček",Jakub Sedmík,NaN,8.0,4.0,0.0,"100,000 .. 200,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5011,game,OMFG: One Million Fatal Guns,1555710,0.0,True,NaN,NaN,['Gun Hit Wonders'],['DigiPen Institute of Technology'],NaN,NaN

In [154]:
app_data = app_data.drop(columns=['price', 'initialprice', 'discount'])

In [101]:
app_data.controller_support = app_data.controller_support.apply(
    lambda x : True if x == 'full' else False
)

In [149]:
app_data_dlc = pd.read_csv('data/download/steam_app_data2.csv')
app_data_dlc = app_data_dlc[['steam_appid', 'dlc']]
app_data_dlc = app_data_dlc.fillna(0)
app_data_dlc

,steam_appid,dlc
0,10,0
1,20,0
2,30,0
3,40,0
4,50,0
...,...,...
4995,1550800,0
4996,1553420,0
4997,1555710,0
4998,1557060,0


In [238]:
app_data[app_data.duplicated(['name_x'])]

,name_x,steam_appid,required_age,is_free,controller_support,developers,publishers,price_overview,platforms,metacritic,genres,release_date,owners_x,positive,negative,userscore,owners_y,average_forever,median_forever,ccu,dlc


In [273]:
genres_set = set()
for gen in app_data.genres:
    for dic in eval(gen):
        genres_set.add(dic['description'])
genres_set

{'Abenteuer',
 'Acción',
 'Accès anticipé',
 'Action',
 'Adventure',
 'Aktion',
 'Animation & Modeling',
 'Audio Production',
 'Aventura',
 'Aventure',
 'Ação',
 'Carreras',
 'Casual',
 'Design & Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Game Development',
 'Gelegenheitsspiele',
 'Gore',
 'Indie',
 'Indépendant',
 'Kostenlos',
 'MMO',
 'Massively Multiplayer',
 'Movie',
 'Multijogador Massivo',
 'Nudity',
 'Photo Editing',
 'RPG',
 'Racing',
 'Rol',
 'Sexual Content',
 'Simulation',
 'Simulação',
 'Simuleringar',
 'Software Training',
 'Sports',
 'Strategi',
 'Strategy',
 'Utilities',
 'Video Production',
 'Violent',
 'Web Publishing',
 'Приключенческие игры',
 'Экшены'}

In [275]:
app_data.genres = app_data.genres.apply(lambda x : eval(x))
app_data

TypeError: eval() arg 1 must be a string, bytes or code object

In [277]:
app_data[app_data.price_overview.isna()]

,name_x,steam_appid,required_age,is_free,controller_support,developers,publishers,price_overview,platforms,metacritic,genres,release_date,owners_x,positive,negative,userscore,owners_y,average_forever,median_forever,ccu,dlc
14,Half-Life 2: Lost Coast,340,0,False,False,['Valve'],['Valve'],NaN,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","27 Oct, 2005",19803000,7872.0,1180.0,0.0,"2,000,000 .. 5,000,000",253.0,24.0,27.0,0
19,Team Fortress 2,440,0,True,False,['Valve'],['Valve'],NaN,"{'windows': True, 'mac': True, 'linux': True}","{'score': 92, 'url': 'https://www.metacritic.c...","[{'id': '1', 'description': 'Action'}, {'id': ...","10 Oct, 2007",51800000,723743.0,45948.0,0.0,"50,000,000 .. 100,000,000",9195.0,354.0,88574.0,1
22,Dota 2,570,0,True,False,['Valve'],['Valve'],NaN,"{'windows': True, 'mac': True, 'linux': True}","{'score': 90, 'url': 'https://www.metacritic.c...","[{'id': '1', 'description': 'Action'}, {'id': ...","9 Jul, 2013",172449000,1247830.0,233051.0,0.0,"100,000,000 .. 200,000,000",34426.0,1130.0,571477.0,2
24,Alien Swarm,630,0,True,False,['Valve'],['Valve'],NaN,"{'windows': True, 'mac': False, 'linux': False}","{'score': 77, 'url': 'https://www.metacritic.c...","[{'id': '1', 'description': 'Action'}]","19 Jul, 2010",4256000,18577.0,1024.0,0.0,"2,000,000 .. 5,000,000",433.0,113.0,91.0,0
25,Counter-Strike: Global Offensive,730,0,True,True,"['Valve', 'Hidden Path Entertainment']",['Valve'],NaN,"{'windows': True, 'mac': True, 'linux': True}","{'score': 83, 'url': 'https://www.metacritic.c...","[{'id': '1', 'description': 'Action'}, {'id': ...","21 Aug, 2012",76611000,4809199.0,647842.0,0.0,"50,000,000 .. 100,000,000",28523.0,6676.0,1079320.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,OpenTTD,1536610,0,True,False,['OpenTTD'],['OpenTTD'],NaN,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '4', 'description': 'Casual'}, {'id': ...","1 Apr, 2021",NaN,1107.0,53.0,0.0,"200,000 .. 500,000",0.0,0.0,2460.0,0
4994,The Tenants - Free Trial,1546540,0,False,False,['Ancient Forge Studio'],['Frozen District'],NaN,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '4', 'description': 'Casual'}, {'id': ...","9 Mar, 2021",NaN,1973.0,127.0,0.0,"200,000 .. 500,000",71.0,73.0,19.0,0
4996,Rifle Strike,1553420,0,True,False,"['Štěpán Sedmík', 'Lukáš Vorlíček']",['Jakub Sedmík'],NaN,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","19 Mar, 2021",110000,8.0,4.0,0.0,"100,000 .. 200,000",0.0,0.0,0.0,0
4997,OMFG: One Million Fatal Guns,1555710,0,True,False,['Gun Hit Wonders'],['DigiPen Institute of Technology'],NaN,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","11 Mar, 2021",NaN,656.0,54.0,0.0,"100,000 .. 200,000",0.0,0.0,55.0,0


In [287]:
app_data[
    (app_data['owners_x'].isna()) &
    (app_data['owners_y'] != '200,000 .. 500,000')
]

,name_x,steam_appid,required_age,is_free,controller_support,developers,publishers,price_overview,platforms,metacritic,genres,release_date,owners_x,positive,negative,userscore,owners_y,average_forever,median_forever,ccu,dlc
1,Team Fortress Classic,20,0,False,False,['Valve'],['Valve'],"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Apr, 1999",NaN,4828.0,821.0,0.0,"2,000,000 .. 5,000,000",49.0,20.0,89.0,0
3,Deathmatch Classic,40,0,False,False,['Valve'],['Valve'],"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Jun, 2001",NaN,1712.0,372.0,0.0,"5,000,000 .. 10,000,000",649.0,8.0,6.0,0
5,Ricochet,60,0,False,False,['Valve'],['Valve'],"{'currency': 'RUB', 'initial': 13300, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Nov, 2000",NaN,3448.0,797.0,0.0,"5,000,000 .. 10,000,000",37.0,10.0,7.0,0
15,Half-Life Deathmatch: Source,360,0,False,False,['Valve'],['Valve'],"{'currency': 'RUB', 'initial': 25900, 'final':...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 May, 2006",NaN,2078.0,674.0,0.0,"5,000,000 .. 10,000,000",554.0,5.0,15.0,0
43,Final DOOM,2290,0,False,False,['id Software'],['id Software'],"{'currency': 'RUB', 'initial': 29900, 'final':...","{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '1', 'description': 'Action'}]","3 Aug, 2007",NaN,871.0,135.0,0.0,"500,000 .. 1,000,000",4.0,6.0,9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4858,STAR WARS™: The Old Republic™,1286830,0,True,False,['BioWare'],['Electronic Arts'],NaN,"{'windows': True, 'mac': False, 'linux': False}",85.0,"[{'id': '37', 'description': 'Free to Play'}, ...","21 Jul, 2020",NaN,21531.0,2325.0,0.0,"2,000,000 .. 5,000,000",1049.0,333.0,6691.0,3
4874,EA SPORTS™ FIFA 21,1313860,0,False,True,['Electronic Arts'],['Electronic Arts'],"{'currency': 'RUB', 'initial': 349900, 'final'...","{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '28', 'description': 'Simulation'}, {'...","8 Oct, 2020",NaN,20296.0,7729.0,0.0,"500,000 .. 1,000,000",6536.0,6856.0,20532.0,0
4891,RuneScape ®,1343400,0,True,False,['Jagex'],['Jagex'],NaN,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '37', 'description': 'Free to Play'}, ...","14 Oct, 2020",NaN,11276.0,1344.0,0.0,"1,000,000 .. 2,000,000",1943.0,42.0,8073.0,4
4933,Krunker,1408720,0,True,False,['Yendis Entertainment Pty'],['Yendis Entertainment Pty'],NaN,"{'windows': True, 'mac': True, 'linux': False}",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","1 Feb, 2021",NaN,3569.0,1078.0,0.0,"500,000 .. 1,000,000",0.0,0.0,1014.0,0


In [291]:
onemore = pd.read_csv('data/download/onemoreround.csv')
onemore

,#,Game,Release date,Price,Score rank(Userscore / Metascore),Owners,Players,Playtime (Median)
0,2,Dota 2,"Jul 9, 2013",Free,N/A (N/A/90%),173 462 000,11%,23:18 (08:52)
1,1,Counter-Strike: Global Offensive,"Aug 21, 2012",Free,N/A (N/A/83%),77 113 000,19%,15:58 (06:34)
2,3,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017","29,99 $",N/A (N/A/86%),53 920 000,9%,12:38 (04:39)
3,20,Team Fortress 2,"Oct 10, 2007",Free,N/A (N/A/92%),51 975 000,3%,18:48 (04:17)
4,15,Unturned,"Jul 7, 2017",Free,N/A (N/A),43 626 000,2%,60:32 (14:55)
...,...,...,...,...,...,...,...,...
387,326,Puyo Puyo Tetris 2,"Mar 23, 2021","29,99 $",N/A (N/A),9 000,100%,02:26 (02:26)
388,219,Empires in Ruins,"Mar 25, 2021","17,99 $",N/A (N/A),6 000,50%,07:35 (03:47)
389,368,Zombie Killin',"Jan 31, 2017","1,99 $",N/A (N/A),4 000,100%,01:35 (01:35)
390,385,Arise: A Simple Story,"Dec 3, 2020","19,99 $",N/A (N/A/79%),4 000,100%,01:20 (01:20)


In [293]:
onemore = onemore[['Game', 'Owners']]
onemore

,Game,Owners
0,Dota 2,173 462 000
1,Counter-Strike: Global Offensive,77 113 000
2,PLAYERUNKNOWN'S BATTLEGROUNDS,53 920 000
3,Team Fortress 2,51 975 000
4,Unturned,43 626 000
...,...,...
387,Puyo Puyo Tetris 2,9 000
388,Empires in Ruins,6 000
389,Zombie Killin',4 000
390,Arise: A Simple Story,4 000


In [370]:
app_data.metacritic = app_data.metacritic.apply(
    lambda x : int(x) - 1 if pd.notnull(x) else np.nan
)

C:\Users\Daniil\miniconda3\envs\mipt-stats\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [397]:
publishers_ind = list()
for devl in app_data.publishers:
    for dev in eval(devl):
        if dev not in publishers_ind:
            publishers_ind.append(dev)
publishers_ind

['Valve',
 'Tripwire Interactive',
 'Introversion Software',
 'Arkane Studios',
 'Topware Interactive',
 'ACE',
 'Ubisoft',
 'id Software',
 'Bethesda Softworks',
 'Bethesda-Softworks',
 'Blazing Griffin',
 'Blazing Griffin Ltd.',
 'Frozenbyte',
 'Bohemia Interactive',
 'Activision',
 'Atari',
 'Aspyr (Mac)',
 'Egosoft',
 'Strategy First',
 '1C Entertainment',
 '1C-SoftClub',
 'Deep Silver',
 'PopCap Games, Inc.',
 'Electronic Arts',
 'Rebellion',
 'Double Fine Productions',
 '2K',
 'GSC Game World',
 'SEGA',
 'Feral Interactive (Mac)',
 'Feral Interactive (Linux)',
 'GSC World Publishing',
 'LucasArts',
 'Lucasfilm',
 'Disney',
 'Klei Entertainment',
 'Funcom',
 'Stunlock Studios',
 'Capcom',
 'Kalypso Media Digital',
 'Io-Interactive A/S',
 'Square Enix',
 'IO Interactive A/S',
 'Topware Interactive, ACE',
 'Telltale Games',
 'CCP',
 'SimBin',
 'Aspyr (Linux)',
 'THQ Nordic',
 'Perfect World Entertainment',
 'Codemasters',
 'Rockstar Games',
 '',
 'U.S. Army',
 'Epic Games, Inc.',
 '

In [411]:
developers_ind

['Valve',
 'Gearbox Software',
 'Hidden Path Entertainment',
 'Tripwire Interactive',
 'Introversion Software',
 'Arkane Studios',
 'Reality Pump Studios',
 'id Software',
 'RavenSoft / id Software',
 'Xatrix Entertainment',
 'Raven Software',
 'Outerlight Ltd.',
 'Frozenbyte',
 'Black Element',
 'Troika Games',
 'Infinity Ward',
 'Gray Matter Studios',
 'Frontier',
 'Aspyr (Mac)',
 'Egosoft',
 'Bugbear Entertainment',
 '1C Entertainment',
 'Katauri Interactive',
 'Mindware Studios',
 'PopCap Games, Inc.',
 'Rebellion',
 'Elixir Studios',
 'Double Fine Productions',
 'Firaxis Games',
 'Facepunch Studios',
 'GSC Game World',
 'Relic Entertainment',
 'CREATIVE ASSEMBLY',
 'Feral Interactive (Mac)',
 'Feral Interactive (Linux)',
 'Monolith',
 'The Creative Assembly',
 'LucasArts',
 'Pandemic Studios',
 'Klei Entertainment',
 'Funcom',
 'Stunlock Studios',
 'CAPCOM Co., Ltd.',
 'Pyro Studios',
 'Io-Interactive A/S',
 'Eidos Interactive',
 'Avalanche Studios',
 'IO Interactive A/S',
 'Ion S

In [403]:
devs = pd.DataFrame(developers_ind)
devs['ind'] = devs.index
pubs = pd.DataFrame(publishers_ind)
pubs['ind'] = pubs.index

In [439]:
devs = devs.set_index(0)

In [437]:
game_pubs = pd.DataFrame(columns=['steam_appid', 'pub_id'])
for row in app_data.iterrows():
    devl = row[1].publishers
    for dev in eval(devl):
        newrow = pd.DataFrame(
            data=[[row[1].steam_appid, pubs.loc[dev].ind]],
            columns=['steam_appid', 'pub_id']
        )
        game_pubs = game_pubs.append(newrow)
game_pubs

,steam_appid,pub_id
0,10,0
0,20,0
0,30,0
0,40,0
0,50,0
...,...,...
0,1506630,1521
0,1516320,1522
0,1533390,1523
0,1536610,1524


In [441]:
game_devs = pd.DataFrame(columns=['steam_appid', 'dev_id'])
for row in app_data.iterrows():
    devl = row[1].developers
    for dev in eval(devl):
        newrow = pd.DataFrame(
            data=[[row[1].steam_appid, devs.loc[dev].ind]],
            columns=['steam_appid', 'dev_id']
        )
        game_devs = game_devs.append(newrow)
game_devs

,steam_appid,dev_id
0,10,0
0,20,0
0,30,0
0,40,0
0,50,1
...,...,...
0,1506630,2247
0,1516320,2248
0,1533390,2249
0,1536610,2250


In [443]:
devs.to_csv('data/download/tab_devs.csv', index=True)
pubs.to_csv('data/download/tab_pubs.csv', index=True)
game_pubs.to_csv('data/download/game_pubs.csv', index=False)
game_devs.to_csv('data/download/game_devs.csv', index=False)

In [444]:
app_data

,name_x,steam_appid,required_age,is_free,controller_support,developers,publishers,price_overview,platforms,metacritic,genres,release_date,positive,negative,average_forever,median_forever,ccu,dlc,copies
0,Counter-Strike,10,0,False,False,['Valve'],['Valve'],259.0,"{'windows': True, 'mac': True, 'linux': True}",88.0,"[{'id': '1', 'description': 'Action'}]","1 Nov, 2000",174744,4487,9273,295,19625,0,18567000
1,Team Fortress Classic,20,0,False,False,['Valve'],['Valve'],133.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Apr, 1999",4828,821,49,20,89,0,4496000
2,Day of Defeat,30,0,False,False,['Valve'],['Valve'],133.0,"{'windows': True, 'mac': True, 'linux': True}",79.0,"[{'id': '1', 'description': 'Action'}]","1 May, 2003",4647,515,954,32,141,0,6159000
3,Deathmatch Classic,40,0,False,False,['Valve'],['Valve'],133.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Jun, 2001",1712,372,649,8,6,0,7063000
4,Half-Life: Opposing Force,50,0,False,False,['Gearbox Software'],['Valve'],133.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Nov, 1999",10834,558,781,139,102,0,8229000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,Minicraft Shooter,1506630,0,False,False,['Sweet Games'],['Sweet Games'],30.0,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","26 Feb, 2021",1,4,0,0,0,0,336000
3219,Skeleton King,1516320,0,False,False,['Forbidden Knowledge'],['Forbidden Knowledge'],30.0,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '4', 'description': 'Casual'}, {'id': ...","26 Jan, 2021",5,1,0,0,0,0,292000
3221,Gorilla Tag,1533390,0,True,False,['Another Axiom'],['Another Axiom'],0.0,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","12 Feb, 2021",5101,124,81,116,272,1,421000
3222,OpenTTD,1536610,0,True,False,['OpenTTD'],['OpenTTD'],0.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '4', 'description': 'Casual'}, {'id': ...","1 Apr, 2021",1107,53,0,0,2460,0,403000


In [448]:
genres_ind = list()
for genl in app_data.genres:
    for gen in genl:
        if gen['description'] not in genres_ind:
            genres_ind.append(gen['description'])
genres_ind

['Action',
 'Free to Play',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Animation & Modeling',
 'Video Production',
 'Ação',
 'Simulation',
 'Racing',
 'Casual',
 'Simulação',
 'Massively Multiplayer',
 'Экшены',
 'Приключенческие игры',
 'Sports',
 'Early Access',
 'Violent',
 'Gore',
 'Design & Illustration',
 'Web Publishing',
 'Aventura',
 'Multijogador Massivo',
 'Nudity',
 'Utilities',
 'Education',
 'Software Training',
 'Game Development',
 'Photo Editing',
 'Aktion',
 'Abenteuer',
 'Gelegenheitsspiele',
 'Kostenlos',
 'MMO',
 'Sexual Content',
 'Audio Production',
 'Movie',
 'Acción',
 'Rol',
 'Simuleringar',
 'Strategi',
 'Aventure',
 'Indépendant',
 'Accès anticipé',
 'Carreras']

In [451]:
genres = pd.DataFrame(genres_ind)
genres['ind'] = genres.index
genres = genres.set_index(0)
genres

,ind
0,
Action,0
Free to Play,1
Strategy,2
Adventure,3
Indie,4
RPG,5
Animation & Modeling,6
Video Production,7
Ação,8


In [454]:
game_genres = pd.DataFrame(columns=['steam_appid', 'genre_id'])
for row in app_data.iterrows():
    genl = row[1].genres
    for gen in genl:
        newrow = pd.DataFrame(
            data=[[row[1].steam_appid, genres.loc[gen['description']].ind]],
            columns=['steam_appid', 'genre_id']
        )
        game_genres = game_genres.append(newrow)
game_genres

,steam_appid,genre_id
0,10,0
0,20,0
0,30,0
0,40,0
0,50,0
...,...,...
0,1536610,4
0,1536610,9
0,1546540,11
0,1546540,4


In [455]:
genres.to_csv('data/download/tab_genres.csv', index=True)
game_genres.to_csv('data/download/game_genres.csv', index=True)

In [459]:
app_data['linux'] = app_data.platforms.apply(
    lambda x : eval(x)['linux']
)
app_data

,name_x,steam_appid,required_age,is_free,controller_support,developers,publishers,price_overview,platforms,metacritic,genres,release_date,positive,negative,average_forever,median_forever,ccu,dlc,copies,windows,mac,linux
0,Counter-Strike,10,0,False,False,['Valve'],['Valve'],259.0,"{'windows': True, 'mac': True, 'linux': True}",88.0,"[{'id': '1', 'description': 'Action'}]","1 Nov, 2000",174744,4487,9273,295,19625,0,18567000,True,True,True
1,Team Fortress Classic,20,0,False,False,['Valve'],['Valve'],133.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Apr, 1999",4828,821,49,20,89,0,4496000,True,True,True
2,Day of Defeat,30,0,False,False,['Valve'],['Valve'],133.0,"{'windows': True, 'mac': True, 'linux': True}",79.0,"[{'id': '1', 'description': 'Action'}]","1 May, 2003",4647,515,954,32,141,0,6159000,True,True,True
3,Deathmatch Classic,40,0,False,False,['Valve'],['Valve'],133.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Jun, 2001",1712,372,649,8,6,0,7063000,True,True,True
4,Half-Life: Opposing Force,50,0,False,False,['Gearbox Software'],['Valve'],133.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]","1 Nov, 1999",10834,558,781,139,102,0,8229000,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,Minicraft Shooter,1506630,0,False,False,['Sweet Games'],['Sweet Games'],30.0,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","26 Feb, 2021",1,4,0,0,0,0,336000,True,False,False
3219,Skeleton King,1516320,0,False,False,['Forbidden Knowledge'],['Forbidden Knowledge'],30.0,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '4', 'description': 'Casual'}, {'id': ...","26 Jan, 2021",5,1,0,0,0,0,292000,True,False,False
3221,Gorilla Tag,1533390,0,True,False,['Another Axiom'],['Another Axiom'],0.0,"{'windows': True, 'mac': False, 'linux': False}",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","12 Feb, 2021",5101,124,81,116,272,1,421000,True,False,False
3222,OpenTTD,1536610,0,True,False,['OpenTTD'],['OpenTTD'],0.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '4', 'description': 'Casual'}, {'id': ...","1 Apr, 2021",1107,53,0,0,2460,0,403000,True,True,True


In [464]:
app_data.to_csv('data/download/all_app_data.csv', index=True)

In [13]:
devst = pd.read_csv('data/download/postgres_games_ent_developers.csv')
pubst = pd.read_csv('data/download/postgres_games_ent_publishers.csv')
countries = pd.read_csv('data/download/countries.csv')

In [14]:
devst = devst.merge(
    countries,
    left_on='Valve',
    right_on='Developer'
)
devst

,0,Valve,Unnamed: 2,Developer,City,Administrative division,Country,Est.,"Notable games, series or franchises",Notes
0,1,Gearbox Software,NaN,Gearbox Software,Plano,Texas,United States,1999,Brothers In Arms seriesBorderlands seriesHalf-...,Acquired by Embracer Group in 2021.
1,3,Tripwire Interactive,NaN,Tripwire Interactive,Roswell,Georgia,United States,2005,Killing Floor seriesRed Orchestra: Ostfront 41-45,NaN
2,4,Introversion Software,NaN,Introversion Software,London,England,United Kingdom,2002,DarwiniaDEFCON,NaN
3,5,Arkane Studios,NaN,Arkane Studios,Lyon,NaN,France,1999,Arx FatalisDeathloopDishonored seriesPrey,Subsidiary of ZeniMax Media
4,6,Reality Pump Studios,NaN,Reality Pump Studios,Kraków,NaN,Poland,1995,Two Worlds series,Division of TopWare Interactive
...,...,...,...,...,...,...,...,...,...,...
207,2110,The Coalition,NaN,The Coalition,Vancouver,NaN,Canada,2010,Gears of War series,Subsidiary of Microsoft Studios
208,2150,Respawn Entertainment,NaN,Respawn Entertainment,"Sherman Oaks, Los Angeles",California,United States,2010,TitanfallApex LegendsStar Wars Jedi: Fallen Order,Acquired by Electronic Arts in 2017.
209,2170,Maxis,NaN,Maxis,Emeryville,California,United States,1987,The Sims seriesSimCity series,Became a subsidiary of Electronic Arts in 1997
210,2202,Playground Games,NaN,Playground Games,Royal Leamington Spa,England,United Kingdom,2009,Forza Horizon seriesFable (2020),Acquired by Microsoft Studios in 2018


In [15]:
pubst = pubst.merge(
    countries,
    left_on='Valve',
    right_on='Developer'
)
pubst

,0,Valve,Unnamed: 2,Developer,City,Administrative division,Country,Est.,"Notable games, series or franchises",Notes
0,1,Tripwire Interactive,NaN,Tripwire Interactive,Roswell,Georgia,United States,2005,Killing Floor seriesRed Orchestra: Ostfront 41-45,NaN
1,2,Introversion Software,NaN,Introversion Software,London,England,United Kingdom,2002,DarwiniaDEFCON,NaN
2,3,Arkane Studios,NaN,Arkane Studios,Lyon,NaN,France,1999,Arx FatalisDeathloopDishonored seriesPrey,Subsidiary of ZeniMax Media
3,6,Ubisoft,NaN,Ubisoft,Montreuil-sous-Bois,NaN,France,1986,Rayman seriesAssassin's Creed seriesJust Dance...,Developer/publisher
4,7,id Software,NaN,id Software,Mesquite,Texas,United States,1991,Wolfenstein seriesDoom seriesQuake series,"Acquired by ZeniMax Media on June 24, 2009"
...,...,...,...,...,...,...,...,...,...,...
89,1310,Survios,NaN,Survios,Los Angeles,California,United States,2013,NaN,NaN
90,1314,Boss Key Productions,NaN,Boss Key Productions,Raleigh,North Carolina,United States,2014,LawBreakers,Founded by Cliff Bleszinski and Arjan Brussee
91,1379,Quantic Dream,NaN,Quantic Dream,Paris,NaN,France,1997,FahrenheitHeavy RainBeyond: Two SoulsDetroit: ...,NaN
92,1402,Tate Multimedia,NaN,Tate Multimedia,Warsaw,NaN,Poland,2002,Urban Trial Freestyle,NaN


In [16]:
pubst.to_csv('data/download/pubs_countries.csv', index=True)
devst.to_csv('data/download/devs_countries.csv', index=True)